## 1. Setup Envorinment

In [ ]:
!pip install transformers==4.21.1
!pip install nlp==0.4.0
!pip install torch==1.7.1
!pip install sentencepiece

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/UrduQA-QG.zip /content/
!unzip /content/UrduQA-QG.zip -d /content/

## 2. Generate Data

In [ ]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!python /content/UrduQA-QG/prepare_data.py --input_path /content/UrduQA-QG --output_path /content/UrduQA-QG/data --tokenizer_path /content/UrduQA-QG/tokenizer --input_train_file train.json --input_eval_file eval.json --max_source_length 1024 --max_target_length 80 --output_train_file train.pt --output_eval_file eval.pt

## 3. Training

In [ ]:
!python /content/UrduQA-QG/run_multi.py --tokenizer_name_or_path /content/UrduQA-QG/tokenizer --output_dir /content/UrduQA-QG/output --train_file_path /content/UrduQA-QG/data/train.pt --valid_file_path /content/UrduQA-QG/data/eval.pt --per_device_train_batch_size 6 --per_device_eval_batch_size 6 --gradient_accumulation_steps 8 --learning_rate 1e-4 --num_train_epochs 50 --seed 42 --do_train --do_eval --eval_steps 100 --logging_steps 100 --evaluation_strategy steps --remove_unused_columns False

Streaming output truncated to the last 5000 lines.
 33% 92/279 [00:03<00:06, 27.07it/s]
 34% 96/279 [00:03<00:06, 28.04it/s]
 35% 99/279 [00:03<00:06, 27.78it/s]
 37% 102/279 [00:03<00:06, 27.04it/s]
 38% 105/279 [00:03<00:07, 22.32it/s]
 39% 108/279 [00:04<00:08, 19.86it/s]
 40% 111/279 [00:04<00:07, 21.41it/s]
 41% 114/279 [00:04<00:07, 22.77it/s]
 42% 117/279 [00:04<00:06, 24.52it/s]
 43% 120/279 [00:04<00:06, 24.23it/s]
 44% 123/279 [00:04<00:06, 24.62it/s]
 45% 126/279 [00:04<00:06, 24.99it/s]
 46% 129/279 [00:04<00:05, 26.11it/s]
 47% 132/279 [00:04<00:05, 26.28it/s]
 48% 135/279 [00:05<00:05, 24.56it/s]
 49% 138/279 [00:05<00:05, 24.56it/s]
 51% 141/279 [00:05<00:05, 25.46it/s]
 52% 144/279 [00:05<00:05, 25.80it/s]
 53% 147/279 [00:05<00:05, 26.16it/s]
 54% 150/279 [00:05<00:04, 26.05it/s]
 55% 153/279 [00:05<00:04, 25.91it/s]
 56% 156/279 [00:05<00:04, 25.09it/s]
 57% 159/279 [00:05<00:04, 25.49it/s]
 58% 162/279 [00:06<00:04, 25.39it/s]
 59% 165/279 [00:06<00:04, 24.33it/s]
 6

In [ ]:
!rm -r /content/UrduQA-QG/output/checkpoint-*
!rm -r /content/UrduQA-QG/output/runs
!zip -r /content/output.zip /content/UrduQA-QG/output
!cp /content/output.zip /content/drive/MyDrive/

## 4. Testing

In [ ]:
import sys
sys.path.insert(1, '/content/UrduQA-QG/')
from pipelines import pipeline
nlp = pipeline("multitask-qa-qg", model="/content/UrduQA-QG/output/")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [ ]:
text = "محمد علی ایک امریکی پیشہ ور باکسر اور سرگرم کارکن تھے۔ عظیم ترین کے نام سے موسوم، انہیں 20 ویں صدی کی سب سے اہم کھیلوں کی شخصیات میں سے ایک سمجھا جاتا ہے، اور اکثر اسے اب تک کے سب سے بڑے ہیوی ویٹ باکسر کے طور پر درجہ دیا جاتا ہے۔ 1999 میں انہیں اسپورٹس الیسٹریٹڈ کی طرف سے سپورٹس مین آف دی سنچری اور بی بی سی کی طرف سے سپورٹس پرسنالٹی آف دی سنچری قرار دیا گیا۔"
nlp(text)

[{'answer': 'عظیم ترین', 'question': 'محمد علی کس نام سے موسوم ہے؟'}]